# BBPSSW Protocol 

In [15]:
#imports
from qutip import *
from qutip_qip.circuit import QubitCircuit
from qutip_qip.operations import Gate, cnot
import numpy as np
from math import pi
import matplotlib.pyplot as plt
from qutip.qip.operations.gates import (rx, ry, rz)
from qutip_qip.operations import (
    Gate, controlled_gate, hadamard_transform)

In [17]:
#CALCULATE THETA VALUE FOR GIVEN FIDELITY
def F_to_theta(F):
    """Returns the angle theta corresponding to an input fidelity F.
    
    Keyword arguments:
    F -- fidelity between 0 and 1
    """
    if F < 0 or F > 1:
        raise Exception('Fidelity must be between 0 and 1.')
    else:
        theta = np.arcsin(2*F - 1)
    
    return theta

In [18]:
#CUSTOM CONTROLLED ROTATION X/Y/Z GATES

def controlled_rx(arg_value):
     # controlled rotation X
     mat = np.zeros((4, 4), dtype=np.complex128)
     mat[0, 0] = mat[1, 1] = 1.
     mat[2:4, 2:4] = rx(arg_value).full()
     return Qobj(mat, dims=[[2, 2], [2, 2]])

def controlled_ry(arg_value):
     # controlled rotation X
     mat = np.zeros((4, 4), dtype=np.complex128)
     mat[0, 0] = mat[1, 1] = 1.
     mat[2:4, 2:4] = ry(arg_value).full()
     return Qobj(mat, dims=[[2, 2], [2, 2]])

def controlled_rz(arg_value):
     # controlled rotation X
     mat = np.zeros((4, 4), dtype=np.complex128)
     mat[0, 0] = mat[1, 1] = 1.
     mat[2:4, 2:4] = rz(arg_value).full()
     return Qobj(mat, dims=[[2, 2], [2, 2]])



In [19]:
#FUNCTION TO CHECK IF A STATE IS A WERNER STATE

def check_werner_r1(rho, tol=1e-6):
    """
    Check whether a 4x4 matrix rho corresponds to the Werner-type state:
    rho_W = r1 * |1><1| + (1-r1)/3 * (I4 - |1><1|)
    where |1> = (|01> - |10>)/sqrt(2).
    In the computational basis {|00>, |01>, |10>, |11>}, the matrix form is:
    rho_W = [[(1-r1)/3, 0, 0, 0],
    [0, (2*r1+1)/6, (1-4*r1)/6, 0],
    [0, (1-4*r1)/6, (2*r1+1)/6, 0],
    [0, 0, 0, (1-r1)/3]]
    Parameters
    ----------
    rho : 2D numpy array (shape (4,4))
    The density matrix to test.
    tol : float
    Numerical tolerance for consistency checks.
    Returns
    -------
    (is_werner, r1_value)
    is_werner : bool
    True if all derived r1 values match within 'tol'.
    r1_value : float or None
    The average r1 if consistent, else None.
    """

    # Corner diagonals (should be (1-r1)/3):

    corner_val = rho[0, 0] # same as rho[3,3] ideally

    # Center diagonals (should be (2*r1 + 1)/6):

    center_diag_val = rho[1, 1] # same as rho[2,2] ideally

    # Off-diagonals in the center block (should be (1 - 4*r1)/6):

    center_offdiag_val = rho[1, 2] # same as rho[2,1] ideally

    # Solve for r1 from each expression:

    # 1) corner_val = (1 - r1)/3 => r1 = 1 - 3 * corner_val

    r1_corner = 1.0 - 3.0 * corner_val

    # 2) center_diag_val = (2*r1 + 1)/6 => 2*r1 + 1 = 6*center_diag_val => r1 = 3*center_diag_val - 0.5

    r1_center_diag = 3.0 * center_diag_val - 0.5

    # 3) center_offdiag_val = (1 - 4*r1)/6 => 1 - 4*r1 = 6*center_offdiag_val => r1 = (1 - 6*center_offdiag_val)/4

    r1_center_offdiag = (1.0 - 6.0 * center_offdiag_val) / 4.0

    # Put them all in a list

    r1_candidates = [r1_corner, r1_center_diag, r1_center_offdiag]

    # Check if they are all close within 'tol'
    if (np.allclose(r1_corner, r1_center_diag, atol=tol) and np.allclose(r1_corner, r1_center_offdiag, atol=tol)):

        # If consistent, return average

        r1_mean = np.mean(r1_candidates)
        return True, r1_mean
    else:
    # Not consistent -> not a Werner state of this specific form
        return False, None



In [ ]:
#CIRCUIT GENERATING WERNER CIRCUIT

werner_circuit = QubitCircuit(N=5)
werner_circuit.user_gates = {"cRX": controlled_rx, "cRY": controlled_ry, "cRZ": controlled_rz}

werner_circuit.add_gate("RY", targets=[0], arg_value= pi/6, style={"showarg": True})
werner_circuit.add_gate("CNOT", controls=[0], targets=[2])
werner_circuit.add_gate("H", targets=[0])
werner_circuit.add_gate("CX", controls=[2], targets=[1])
werner_circuit.add_gate("X", targets=[2])
werner_circuit.add_gate("cRX", targets=[2,1], arg_value=pi/2)
werner_circuit.add_gate("cRY", targets=[2,1], arg_value=pi/2)
werner_circuit.add_gate("cRZ", targets=[2,1], arg_value=pi/2)
werner_circuit.add_gate("X", targets=[2])
werner_circuit.add_gate("TOFFOLI", controls=[2,0], targets=[1])
werner_circuit.add_gate("X", targets=[2])
werner_circuit.add_gate("TOFFOLI", controls=[0,2], targets=[3])
werner_circuit.add_gate("TOFFOLI", controls=[1,2], targets=[4])

werner_circuit.draw("matplotlib", dpi=200)

In [21]:
#RUN WERNER CIRCUIT
from qutip import tensor, basis
initial_state =  tensor(basis(2,0), basis(2,0), basis(2,0), basis(2,0), basis(2,0))
result = werner_circuit.run(state=initial_state)

# print(result)
print(tensor(result.ptrace(0), result.ptrace(1)))
print(result.ptrace([0,1]))
print(check_werner_r1(result.ptrace([0,1])))

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.25 0.   0.   0.  ]
 [0.   0.25 0.   0.  ]
 [0.   0.   0.25 0.  ]
 [0.   0.   0.   0.25]]
Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.23325318  0.          0.          0.        ]
 [ 0.          0.26674682 -0.03349365  0.        ]
 [ 0.         -0.03349365  0.26674682  0.        ]
 [ 0.          0.          0.          0.23325318]]
(True, (0.3002404735808355+0j))
